In [1]:
import pandas as pd
import getpass
from neo4j import GraphDatabase

pd.set_option("max_colwidth", 999)
pd.set_option("max_rows", 999)

In [2]:
# Define connection uri, username & password parameters to establish driver connection (using neo4j official driver)
uri = 'neo4j://0.0.0.0:7687'
user = 'neo4j'
password = getpass.getpass("Please enter password to Neo4j Graph: ") # getpass2

Please enter password to Neo4j Graph: ········


### Delete Orphan Nodes Prior to Indexing

In [3]:
# Define driver
driver = GraphDatabase.driver(uri=uri, auth=(user, password))
print(driver)
# driver.close()

In [ ]:
# Test connection
with driver.session(database="neo4j") as session:
    display(session.run("match (n:Code) WHERE n.name contains 'Coronavirus' RETURN n.name").data())

In [ ]:
# with driver.session(database="neo4j") as session:
#     display(session.run("MATCH (attribute:Attribute) WHERE size((attribute)--())=0 DELETE attribute").consume().counters)
    

In [ ]:
# with driver.session(database="neo4j") as session:
#     display(session.run("MATCH (atom:Atom) WHERE size((atom)--())=0 DELETE atom").consume().counters)

### Create Unique Property Constraints & Indexes

Note: Contraints must be applied s/p import of data when utilizing `neo4j-admin import` tool

In [4]:
# Apply all unique property contraints & indexes to the graph
with driver.session(database="neo4j") as session:
    display(session.run("CREATE CONSTRAINT ON (aui:Atom) ASSERT aui.AtomId IS UNIQUE").consume().counters)
    display(session.run("CREATE CONSTRAINT ON (attribute:Attribute) ASSERT attribute.AttributeId IS UNIQUE").consume().counters)
    display(session.run("CREATE CONSTRAINT ON (cui:Concept) ASSERT cui.ConceptId IS UNIQUE").consume().counters)
    display(session.run("CREATE CONSTRAINT ON (code:Code) ASSERT code.SourceCodeId IS UNIQUE").consume().counters)
    display(session.run("CREATE CONSTRAINT ON (atc:ATC) ASSERT atc.SourceCodeId IS UNIQUE").consume().counters)
    display(session.run("CREATE CONSTRAINT ON (go:GO) ASSERT go.SourceCodeId IS UNIQUE").consume().counters)
    display(session.run("CREATE CONSTRAINT ON (hgnc:HGNC) ASSERT hgnc.SourceCodeId IS UNIQUE").consume().counters)
    display(session.run("CREATE CONSTRAINT ON (hpo:HPO) ASSERT hpo.SourceCodeId IS UNIQUE").consume().counters)
    display(session.run("CREATE CONSTRAINT ON (icdo:ICDO3) ASSERT icdo.AttributeId IS UNIQUE").consume().counters)
    display(session.run("CREATE CONSTRAINT ON (icd10cm:ICD10CM) ASSERT icd10cm.SourceCodeId IS UNIQUE").consume().counters)
    display(session.run("CREATE CONSTRAINT ON (icd10pcs:ICD10PCS) ASSERT icd10pcs.SourceCodeId IS UNIQUE").consume().counters)
    display(session.run("CREATE CONSTRAINT ON (lnc:LNC) ASSERT lnc.SourceCodeId IS UNIQUE").consume().counters)
    display(session.run("CREATE CONSTRAINT ON (mdr:MDR) ASSERT mdr.SourceCodeId IS UNIQUE").consume().counters)
    display(session.run("CREATE CONSTRAINT ON (msh:MSH) ASSERT msh.SourceCodeId IS UNIQUE").consume().counters)
    display(session.run("CREATE CONSTRAINT ON (medrt:`MED-RT`) ASSERT medrt.SourceCodeId IS UNIQUE").consume().counters)
    display(session.run("CREATE CONSTRAINT ON (ncbi:NCBI) ASSERT ncbi.SourceCodeId IS UNIQUE").consume().counters)
    display(session.run("CREATE CONSTRAINT ON (ndc:NDC) ASSERT ndc.AttributeId IS UNIQUE").consume().counters)
    display(session.run("CREATE CONSTRAINT ON (nci:NCI) ASSERT nci.SourceCodeId IS UNIQUE").consume().counters)
    display(session.run("CREATE CONSTRAINT ON (omim:OMIM) ASSERT omim.SourceCodeId IS UNIQUE").consume().counters)
    display(session.run("CREATE CONSTRAINT ON (pdq:PDQ) ASSERT pdq.SourceCodeId IS UNIQUE").consume().counters)
    display(session.run("CREATE CONSTRAINT ON (rxnorm:RXNORM) ASSERT rxnorm.SourceCodeId IS UNIQUE").consume().counters)
    display(session.run("CREATE CONSTRAINT ON (sct:SNOMEDCT_US) ASSERT sct.SourceCodeId IS UNIQUE").consume().counters)
    display(session.run("CREATE CONSTRAINT ON (src:SRC) ASSERT src.SourceCodeId IS UNIQUE").consume().counters)
    display(session.run("CREATE CONSTRAINT ON (sty:SemanticType) ASSERT sty.SemanticTypeId IS UNIQUE").consume().counters)
    display(session.run("CREATE CONSTRAINT ON (sty:SemanticType) ASSERT sty.sty IS UNIQUE").consume().counters)
    display(session.run("CREATE CONSTRAINT ON (sty:SemanticType) ASSERT sty.stn IS UNIQUE").consume().counters)
    display(session.run("CREATE INDEX FOR (code:Code) ON (code.vocab)").consume().counters)
    display(session.run("CREATE INDEX FOR (atom:Atom) ON (atom.tty)").consume().counters)
    display(session.run("CREATE INDEX FOR (code:Code) ON (code.code)").consume().counters)
#     display(session.run("CREATE INDEX FOR (cui:Concept) ON (cui.name)").consume().counters)

{'constraints_added': 1}

{'constraints_added': 1}

{'constraints_added': 1}

{'constraints_added': 1}

{'constraints_added': 1}

...

{'constraints_added': 1}

{'constraints_added': 1}

{'indexes_added': 1}

{'indexes_added': 1}

{'indexes_added': 1}

# All Labels/Nodes in Graph

- Note: Labels & nodes have been created in a redundant way in-order to handle individual ontologies/vocabularies (i.e. SNOMEDCT_US, NCI HGNC) as labels.
  - This design implementation allows use of `Code` label (which encompasses all discrete ontologies/vocabularies in the graph - in other words all the other labels except a handful). This enables a broader and more granular approach to querying the graph.
    - For example, if the goal is to create a SNOMEDCT_US -> ICD10CM crosswalk... It is more intuitive, precise & less time intensive to traverse the graph explicitly based on those labels.
    - If we flip the scenario, where it is uncertain what crosswalk(s) may exist between any of the vocabularies, then the `Code` label serves as a "catch-all" label for all vocabularies/ontologies within the graph. Thus preventing tedious trial & error to establish what relationships may exist between certain vocabularies.

- **In the next .ipynb (part2) we will explore this in more detail**


In [5]:
result = {"label": [], "count": []}
with driver.session(database="neo4j") as session:
    for row in session.run("CALL db.labels()"):
        label = row["label"]
        query = f"MATCH (:`{label}`) RETURN count(*) as count"
        count = session.run(query).single()["count"]
        result["label"].append(label)
        result["count"].append(count)
nodes_df = pd.DataFrame(data=result)
nodes_df.sort_values(by="count",ascending=False)

,label,count
1,Atom,6311689
2,Code,3407786
0,Concept,3295170
13,NCBI,1898768
3,MSH,328010
4,SNOMEDCT_US,299049
20,Attribute,273003
21,NDC,247545
5,LNC,231977
17,ICD10PCS,190609


# All Relationships/Edges in the Graph

- Should investigate latter half of list (extremely how counts)
  - If cannot resolve then possibly best to trim where not providing any value


In [6]:
result = {"relType": [], "count": []}
with driver.session(database="neo4j") as session:
    for row in session.run("CALL db.relationshipTypes() YIELD relationshipType RETURN relationshipType"):
        relationship_type = row["relationshipType"]
        query = f"MATCH ()-[:`{relationship_type}`]->() RETURN count(*) as count"
        count = session.run(query).single()["count"]
        result["relType"].append(relationship_type)
        result["count"].append(count)
rels_df = pd.DataFrame(data=result)
rels_df.sort_values(by="count",ascending=False)

,relType,count
2,HAS_UMLS_ATOM,5332842
3,HAS_CUI,5226222
0,HAS_STY,3492268
1,IS_STY_OF,3492268
4,HAS_AUI,3295170
5,HAS_CHILD,2363958
10,CHD,1947562
15,PAR,1947562
529,ATTRIBUTE_HAS_AUI,1204869
13,ISA,622292


In [7]:
# Close connection
driver.close()

##### Establishing a connection using py2neo
To be covered later

In [ ]:
## py2neo connection (community supported python driver)

# from py2neo import Graph, Relationship, Subgraph, database

# g = Graph(uri=uri, auth=(user, password))


In [ ]:
# result = {"label": [], "count": []}
# for label in g.run("CALL db.labels()").to_series():
#     query = f"MATCH (:`{label}`) RETURN count(*) AS count"
#     count = g.run(query).to_data_frame().iloc[0]['count']
#     result["label"].append(label)
#     result["count"].append(count)
# nodes_df = pd.DataFrame(data=result)
# nodes_df.sort_values(by="count", ascending=False)


In [ ]:
# class Neo4jConnection:
    
#     def __init__(self, uri, username, password):
#         self.__uri = uri
#         self.__username = username
#         self.__password = password
#         self.__driver = None
        
#         try:
#             self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__username, self.__password))
#         except Exception as e:
#             print("Failed to create driver", e)
            
#     def close(self):
#         if self.__driver is not None:
#             self.__driver.close()
            
#     def query(self, query, parameters=None, db=None):
#         assert self.__driver is not None, "Driver not initialized."
#         session = None
#         response = None
        
#         try:
#             session = self.__driver.session(database=db) if db is not None else self.__driver.session()
#             response = list(session.run(query, parameters))
#         except Exception as e:
#             print("Query has failed:", e)
#         finally:
#             if session is not None:
#                 session.close()
#         return response

In [ ]:
# # py2neo connection (community supported python driver)

# from py2neo import Graph

# g = Graph(uri=uri, auth=(user, password))
# test = '''
#         MATCH (a:SNOMEDCT_US)-[:HAS_UMLS_ATOM]->(b:Atom)-[:HAS_CUI]->(c:Concept)
#         <-[:HAS_CUI]-(d:Atom)<-[:HAS_UMLS_ATOM]-(e:HPO) 
#         WHERE EXISTS((c)-[:HAS_STY]->(:SemanticType {sty: 'Neoplastic Process'})) 
#         RETURN DISTINCT a.vocab, a.code, a.name, c.ConceptId, c.name, e.vocab, e.code, e.name
#         '''
# test_df = g.run(test).to_data_frame().drop_duplicates(['c.ConceptId']).replace(np.nan, '')
# test_df.head(10)